In [1]:
# packages
import salabim as sim
import numpy as np
import scipy.stats as st
import pandas as pd


In [2]:
# simulation code for MMc queue
def MMc(
    arrival_rate,
    processing_rate,
    charging_stations=1,
    sim_time=50000,
    time_unit="minutes",
    random_seed='*',
):
    # Generator which creates cars
    class CarGenerator(sim.Component):
        # setup method is called when the component is created
        # and is used to initialize the component
        # switch off monitoring for mode and status
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)

        def process(self):
            while True:
                Car(name="Car")
                self.hold(iat_distr.sample())

    class Car(sim.Component):
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)

        def process(self):
            self.enter(waitingline)
            for ChargingStation in ChargingStations:
                if ChargingStation.ispassive():
                    ChargingStation.activate()
                    break  # activate at most one charging station
            self.passivate()

    class ChargingStation(sim.Component):
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)

        def process(self):
            while True:
                while len(waitingline) == 0:
                    self.passivate()
                self.car = waitingline.pop()
                self.hold(srv_distr.sample())
                self.car.activate()

    N_STATION = charging_stations
    iat_distr = sim.Exponential(1 / arrival_rate)
    srv_distr = sim.Exponential(1 / processing_rate)

    # https://www.salabim.org/manual/Reference.html#environment
    app = sim.App(
        trace=False,  # defines whether to trace or not
        random_seed=random_seed,  # if “*”, a purely random value (based on the current time)
        time_unit=time_unit,  # defines the time unit used in the simulation
        name="Charging Station",  # name of the simulation
        do_reset=True,  # defines whether to reset the simulation when the run method is called
        yieldless=True,  # defines whether the simulation is yieldless or not
    )

    # Instantiate and activate the client generator
    CarGenerator(name="Electric Cars Generator")

    # Create Queue and set monitor to stats_only
    # https://www.salabim.org/manual/Queue.html
    waitingline = sim.Queue(name="Waiting Cars", monitor=True)
    # waitingline.length_of_stay.monitor(value=True)
    waitingline.length.reset_monitors(stats_only=True)
    waitingline.length_of_stay.reset_monitors(stats_only=True)

    # Instantiate the servers, list comprehension but only 1 server
    ChargingStations = [ChargingStation() for _ in range(N_STATION)]

    # Execute Simulation
    app.run(till=sim_time)

    # Return results
    return pd.DataFrame(
        {
            "Wq": [waitingline.length_of_stay.mean()],
            "Lq": [waitingline.length.mean()],
        }
    )

  

In [4]:
# Run simulation till infinity with different random seeds

print(
    MMc(
        arrival_rate          = 40/60,    # cars per hour
        processing_rate       = 50/60,    # cars per hour
        charging_stations     = 1,        # charging station
        # sim_time              = 10,       # use this line to test
        sim_time              = sim.inf,  # https://www.salabim.org/manual/Miscellaneous.html#run-control
    ).to_string(index=False)
)

KeyboardInterrupt: 